# Interpret Models

You can use Azure Machine Learning to interpret a model by using an *explainer* that quantifies the amount of influence each feature contribues to the predicted label. There are many common explainers, each suitable for different kinds of modeling algorithm; but the basic approach to using them is the same.

## Install SDK packages

In addition to the latest version of the **azureml-sdk** and **azureml-widgets** packages, you'll need the **azureml-explain-model** package to run the code in this notebook. You'll also use the Azure ML Interpretability library (**azureml-interpret**). You can use this to interpret many typical kinds of model, even if they haven't been trained in an Azure ML experiment or registered in an Azure ML workspace.

Run the cell below to verify that these packages are installed. 

In [1]:
pip show azureml-explain-model azureml-interpret

Name: azureml-explain-model
Version: 1.40.0
Summary: The package has been deprecated and might not receive future updates.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: azureml-interpret
Required-by: 
---
Name: azureml-interpret
Version: 1.40.0
Summary: Machine Learning interpret package is used to interpret ML models
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: azureml-core, numpy, shap, interpret-community, numba
Required-by: azureml-train-automl-runtime, azureml-responsibleai, azureml-explain-model
Note: you may need to restart the kernel to use updated packages.


## Explain a model

Let's start with a model that is trained outside of Azure Machine Learning - Run the cell below to train a decision tree classification model.

In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# Separate features and labels
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

print('Model trained.')

Loading Data...
Training a decision tree model
Accuracy: 0.891
AUC: 0.878344693765859
Model trained.


The training process generated some model evaluation metrics based on a hold-back validation dataset, so you have an idea of how accurately it predicts; but how do the features in the data influence the prediction?

### Get an explainer for the model

Let's get a suitable explainer for the model from the Azure ML interpretability library you installed earlier. There are many kinds of explainer. In this example you'll use a *Tabular Explainer*, which is a "black box" explainer that can be used to explain many kinds of model by invoking an appropriate [SHAP](https://github.com/slundberg/shap) model explainer.

In [3]:
from interpret.ext.blackbox import TabularExplainer

# "features" and "classes" fields are optional
tab_explainer = TabularExplainer(model,
                             X_train, 
                             features=features, 
                             classes=labels)
print(tab_explainer, "ready!")

TabularExplainer ready!


### Get *global* feature importance

The first thing to do is try to explain the model by evaluating the overall *feature importance* - in other words, quantifying the extent to which each feature influences the prediction based on the whole training dataset.

In [4]:
# you can use the training data or the test data here
global_tab_explanation = tab_explainer.explain_global(X_train)

# Get the top features by importance
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
for feature, importance in global_tab_feature_importance.items():
    print(feature,":", importance)

Pregnancies : 0.21964759484373805
Age : 0.10556241519182138
BMI : 0.0924192845549131
SerumInsulin : 0.06851238465356388
PlasmaGlucose : 0.049605728827745485
TricepsThickness : 0.020985885113385568
DiastolicBloodPressure : 0.017048777120842632
DiabetesPedigree : 0.013867836946051785


The feature importance is ranked, with the most important feature listed first.

### Get *local* feature importance

So you have an overall view, but what about explaining individual observations? Let's generate *local* explanations for individual predictions, quantifying the extent to which each feature influenced the decision to predict each of the possible label values. In this case, it's a binary model, so there are two possible labels (non-diabetic and diabetic); and you can quantify the influence of each feature for each of these label values for individual observations in a dataset. You'll just evaluate the first two cases in the test dataset.

In [5]:
# Get the observations we want to explain (the first two)
X_explain = X_test[0:2]

# Get predictions
predictions = model.predict(X_explain)

# Get local explanations
local_tab_explanation = tab_explainer.explain_local(X_explain)

# Get feature names and importance for each possible label
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])

Support for not-diabetic
	Observation 1
		 SerumInsulin : 0.3724896113852634
		 Age : 0.23527096475599307
		 TricepsThickness : 0.02542878747577428
		 BMI : 0.01319756466947267
		 DiabetesPedigree : -0.0005625606028256819
		 DiastolicBloodPressure : -0.018434660282536524
		 PlasmaGlucose : -0.03635770194423372
		 Pregnancies : -0.2576034340283365
		 ----------
		 Total: 0.33342857142857096 Prediction: not-diabetic
	Observation 2
		 BMI : 0.3480614924346044
		 Age : 0.043768352744836474
		 Pregnancies : 0.017397564077217806
		 DiabetesPedigree : 0.01509439335772088
		 PlasmaGlucose : -0.0031480016919270123
		 DiastolicBloodPressure : -0.013288111191301495
		 TricepsThickness : -0.02140921294575149
		 SerumInsulin : -0.053047905356828746
		 ----------
		 Total: 0.3334285714285708 Prediction: not-diabetic
Support for diabetic
	Observation 1
		 Pregnancies : 0.2576034340283365
		 PlasmaGlucose : 0.03635770194423378
		 DiastolicBloodPressure : 0.01843466028253655
		 DiabetesPedigree : 0.000

## Adding explainability to a model training experiment

As you've seen, you can generate explanations for models trained outside of Azure Machine Learning; but when you use experiments to train and register models in your Azure Machine Learning workspace, you can generate model explanations and log them.

Run the code in the following cell to connect to your workspace.

> **Note**: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [6]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.40.0 to work with ss_my_aml


### Train and explain a model using an experiment

OK, let's create an experiment and put the files it needs in a local folder - in this case we'll just use the same CSV file of diabetes data to train the model.

In [7]:
import os, shutil
from azureml.core import Experiment

# Create a folder for the experiment files
experiment_folder = 'diabetes_train_and_explain'
os.makedirs(experiment_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(experiment_folder, "diabetes.csv"))

'diabetes_train_and_explain/diabetes.csv'

Now we'll create a training script that looks similar to any other Azure ML training script except that is includes the following features:

- The same libraries to generate model explanations we used before are imported and used to generate a global explanation
- The **ExplanationClient** library is used to upload the explanation to the experiment output

In [8]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Import Azure ML run library
from azureml.core.run import Run

# Import libraries for model explanation
from azureml.interpret import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# Separate features and labels
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes.pkl')

# Get explanation
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# Get an Explanation Client and upload the explanation
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')

# Complete the run
run.complete()

Writing diabetes_train_and_explain/diabetes_training.py


The experiment needs a Python environment in which to run the script, so we'll define a Conda specification for it. Note that the **azureml-interpret** library is included in the training environment so the script can create a **TabularExplainer** and use the **ExplainerClient** class.

In [9]:
%%writefile $experiment_folder/interpret_env.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pandas
- pip
- pip:
  - azureml-defaults
  - azureml-interpret

Writing diabetes_train_and_explain/interpret_env.yml


Now you can run the experiment.

In [10]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails


# Create a Python environment for the experiment
explain_env = Environment.from_conda_specification("explain_env", experiment_folder + "/interpret_env.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                      script='diabetes_training.py',
                      environment=explain_env,
                      docker_runtime_config=DockerConfiguration(use_docker=True)) 

# submit the experiment
experiment_name = 'mslearn-diabetes-explain'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'mslearn-diabetes-explain_1652099227_e915db10',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-05-09T12:29:17.989678Z',
 'endTimeUtc': '2022-05-09T12:29:25.088555Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '2b808d49-fc88-4e8e-99a7-9cb0f91291ca',
  'azureml.git.repository_uri': 'https://github.com/MicrosoftLearning/mslearn-dp100',
  'mlflow.source.git.repoURL': 'https://github.com/MicrosoftLearning/mslearn-dp100',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '2bae8ae19514c40f36500ab5b6e0ce2e1c84f4cd',
  'mlflow.source.git.commit': '2bae8ae19514c40f36500ab5b6e0ce2e1c84f4cd',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'ta

## Retrieve the feature importance values

With the experiment run completed, you can use the **ExplanationClient** class to retrieve the feature importance from the explanation registered for the run.

In [11]:
from azureml.interpret import ExplanationClient

# Get the feature explanations
client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

Feature	Importance
Pregnancies 	 0.2225959591056461
Age 	 0.10391516011846007
BMI 	 0.09743871457454463
SerumInsulin 	 0.07091799284316275
PlasmaGlucose 	 0.04848554443223309
TricepsThickness 	 0.021256531991758747
DiastolicBloodPressure 	 0.018261899925665008
DiabetesPedigree 	 0.01562952761627036


## View the model explanation in Azure Machine Learning studio

You can also click the **View run details** link in the Run Details widget to see the run in Azure Machine Learning studio, and view the **Explanations** tab. Then:

1. Select the explanation ID for your tabular explainer.
2. View the **Aggregate feature importance** chart, which shows the overall global feature importance.
3. View the **Individual feature importance** chart, which shows each data point from the test data.
4. Select an individual point to see the local feature importance for the individual prediction for the selected data point.
5. Use the **New Cohort** button to define a subset of the data with the following settings:
    - **Dataset cohort name**: Under 25s
    - **Select filter**: Dataset
        - Age less than 25 (Make sure you add this filter before saving the new cohort).
6. Create a second new cohort named **25 and over** with a filter on Age greater than or equal to 25.
6. Review the **Aggregate feature importance** visualization and compare the relative feature importance for the two cohorts you have defined. The ability to compare cohorts makes it possible to see how the features influence preedictions differently for multiple subsets of the data population.



**More Information**: For more information about using explainers in Azure ML, see [the documentation](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability). 